In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Attention(Q,K,V) = softmax[(Q.K')/root(d_k)].V

### Q : Query - what i am looking for
### K : Key - what others offer
### V : Value - actual information

In [13]:
import numpy as np

In [14]:
#no. of tokens on sequence
sequence_len = 4

#embedding dimension
d_model = 6

#input embeddings
x = np.random.rand(sequence_len, d_model)
print(x)
print(f"\nShape : {x.shape}")

[[0.93005719 0.96356597 0.3899102  0.11549956 0.02060023 0.68037755]
 [0.57644196 0.07512759 0.92964628 0.15802323 0.32185567 0.53133435]
 [0.78743228 0.30391787 0.55046969 0.34859357 0.06228382 0.66622843]
 [0.90800795 0.46334996 0.4132689  0.58411751 0.94683309 0.26074614]]

Shape : (4, 6)


## Create Q,K,V Matrix

In [15]:
#for easy understanding
d_k = d_model

#creating Q,K,V Matrix
W_Q = np.random.rand(d_model,d_k)
W_K = np.random.rand(d_model,d_k)
W_V = np.random.rand(d_model,d_k)

Q = x @ W_Q
K = x @ W_K
V = x @ W_V

print(f"\nQ : \n{Q} \nShape : {Q.shape}")
print(f"\nK : \n{K} \nShape : {K.shape}")
print(f"\nV : \n{V} \nShape : {V.shape}")


Q : 
[[1.75975711 1.67375474 1.7919927  1.50254964 1.40171025 0.90861703]
 [1.19211002 1.39716365 1.52648878 1.81139571 1.39100199 0.69884426]
 [1.44133759 1.46221427 1.76738649 1.58606809 1.49271479 0.89281604]
 [1.44023694 1.78673759 1.88090438 2.35608502 2.09163042 1.06515897]] 
Shape : (4, 6)

K : 
[[1.15688769 2.11656521 1.39406858 1.14075957 1.4329303  1.21591145]
 [1.18293831 1.47261021 0.77599073 1.41323644 1.61694652 1.00820699]
 [0.91751487 1.88151508 1.02606015 1.20258919 1.44054652 1.03619253]
 [1.58668753 2.11291726 1.96981073 0.8054734  2.13099095 1.1409964 ]] 
Shape : (4, 6)

V : 
[[1.49484254 1.64077782 1.41324377 1.80463794 1.64963664 2.37049496]
 [1.43574293 1.79218653 0.74631516 0.93136138 1.20471444 2.11582077]
 [1.30394543 1.60438506 1.08740763 1.32236623 1.2281343  2.20230381]
 [1.94358188 1.99256734 0.9887631  1.62996855 2.00536579 2.49385773]] 
Shape : (4, 6)


## Calculate Score - Q.K'

In [16]:
print("Shape of Q.K' : ")
print(f"{Q.shape} x {K.shape[::-1]} --> (4,4)")

#calculate score
score = Q @ K.T  #K.T as we take the transpose of matrix K

print("\nScore : \n",score)
print("\nShape : ",score.shape)

Shape of Q.K' : 
(4, 6) x (6, 4) --> (4,4)

Score : 
 [[12.90401193 11.24306459 11.37017133 15.09260076]
 [11.37366407 10.16589713 10.19513762 13.07111377]
 [12.26005941 10.78503323 10.86991446 14.33510428]
 [15.05007485 13.58010428 13.56333661 17.33581217]]

Shape :  (4, 4)


## Scale the score - (Q.K')/root(d_k)

In [17]:
scaled_score = score/np.sqrt(d_k)
print("Scaled score : \n",scaled_score)

Scaled score : 
 [[5.26804081 4.5899619  4.64185301 6.16152846]
 [4.64327891 4.15021012 4.16214751 5.33625985]
 [5.0051483  4.40297138 4.437624   5.85228182]
 [6.14416733 5.54405436 5.53720899 7.07731568]]


## Apply Softmax - softmax[(Q.K')/root(d_k)]

In [18]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x/np.sum(exp_x, axis=1, keepdims=True)

In [19]:
attention_weights = softmax(scaled_score)

print(f"Attention Weights : \n {attention_weights}")
print("Shape : ",attention_weights.shape)

Attention Weights : 
 [[0.22292297 0.11315373 0.11918042 0.54474289]
 [0.23649045 0.14443637 0.14617089 0.47290228]
 [0.22484565 0.12312958 0.12747113 0.52455364]
 [0.21569124 0.11836049 0.11755303 0.54839524]]
Shape :  (4, 4)


## Weighted sum - W.V

In [20]:
attention_output = attention_weights @ V
print(f"Attention Output : \n{attention_output}")
print("Shape : ",attention_output.shape)

Attention Output : 
[[1.70985156 1.84520781 1.0677122  1.55319619 1.74283813 2.38883353]
 [1.67061266 1.82368925 1.06854939 1.52540941 1.69198745 2.36746463]
 [1.69861961 1.83931416 1.06692755 1.54401258 1.7277232  2.38240791]
 [1.71149346 1.84734025 1.06321957 1.54879614 1.74250673 2.38823178]]
Shape :  (4, 6)


## Full Function for complete process

In [21]:
def self_attention(x):
    d_model = x.shape[1]
    
    W_Q = np.random.rand(d_model, d_model)
    W_K = np.random.rand(d_model, d_model)
    W_V = np.random.rand(d_model, d_model)
    
    Q = x @ W_Q
    K = x @ W_K
    V = x @ W_V
    
    scores = Q @ K.T
    scaled_scores = scores / np.sqrt(d_model)
    
    attention_weights = softmax(scaled_scores)
    output = attention_weights @ V
    
    return attention_weights,output

In [22]:
seq_len = 5
d_model = 8
x = np.random.rand(seq_len, d_model)

attention_weights, output = self_attention(x)

print(f"\nAttention weights : \n{attention_weights}")
print("Shape : ",attention_weights.shape)

print(f"\nOutput : \n{output}")
print("Shape : ",output.shape)


Attention weights : 
[[2.71482664e-03 3.74518890e-02 7.82541984e-01 9.04945649e-02
  8.67967360e-02]
 [2.44648819e-04 1.08083630e-02 9.16287033e-01 3.59279424e-02
  3.67320125e-02]
 [8.30035002e-05 5.97854869e-03 9.45038242e-01 2.26042844e-02
  2.62959214e-02]
 [3.97036028e-04 1.34191676e-02 8.91136290e-01 4.60037197e-02
  4.90437863e-02]
 [3.87672542e-04 1.27850822e-02 9.03072198e-01 3.79007917e-02
  4.58542555e-02]]
Shape :  (5, 5)

Output : 
[[3.15014839 2.17171424 2.84984266 2.04070751 2.31609572 2.53463607
  2.79892322 2.29536214]
 [3.25496955 2.1831258  2.89529331 2.05594376 2.33872345 2.63328027
  2.84698534 2.38478055]
 [3.27728613 2.18472314 2.90401572 2.05896955 2.34328362 2.65437486
  2.85690996 2.40367203]
 [3.2349348  2.18183509 2.88776381 2.05252535 2.33375514 2.61463335
  2.83799806 2.3668183 ]
 [3.24396888 2.18148192 2.89024231 2.05357848 2.33530882 2.62336143
  2.84160933 2.37406446]]
Shape :  (5, 8)
